# Naive Bayes (the easy way)

We'll cheat by using sklearn.naive_bayes to train a spam classifier! Most of the code is just loading our training data into a pandas DataFrame that we can play with:

In [33]:
import os
import io
import numpy
import pandas as pd
from pandas import DataFrame
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

def readFiles(path):
    for root, dirnames, filenames in os.walk(path):
        for filename in filenames:
            path = os.path.join(root, filename)

            inBody = False
            lines = []
            f = io.open(path, 'r', encoding='latin1')
            for line in f:
                if inBody:
                    lines.append(line)
                elif line == '\n':
                    inBody = True
            f.close()
            message = '\n'.join(lines)
            yield path, message


def dataFrameFromDirectory(path, classification):
    rows = []
    index = []
    for filename, message in readFiles(path):
        rows.append({'message': message, 'class': classification})
        index.append(filename)

    return DataFrame(rows, index=index)

data = DataFrame({'message': [], 'class': []})

data = pd.concat([data, dataFrameFromDirectory("emails/spam", "spam")]);
data = pd.concat([data, dataFrameFromDirectory("emails/ham", "ham")])

#For Pandas 1.3:
#data = data.append(dataFrameFromDirectory('emails/spam', 'spam'))
#data = data.append(dataFrameFromDirectory('emails/ham', 'ham'))


Let's have a look at that DataFrame:

In [34]:
data.tail()

,message,class
emails/ham/01037.6b42b5f3d3d9e6293bf24af66b250655,"On Thu, 2002-08-29 at 00:02, Ville Skyttä wrot...",ham
emails/ham/02056.7bc7703e40a24dda665d4ce7b0cba710,"URL: http://www.newsisfree.com/click/-6,835599...",ham
emails/ham/01782.278f53b8f65fcd422cb26c5bbe74599d,use Perl Daily Newsletter\n\n\n\nIn this issue...,ham
emails/ham/00043.d2673a72d215cbdd747dc98cde41fbd2,< >\n\n> I downloaded a driver from the nVidia...,ham
emails/ham/00135.bd3bc1c036eab89c9c50cff40958c939,>That Goddess Chick wrote:\n\n>>\n\n>> >Thank...,ham


Now we will use a CountVectorizer to split up each message into its list of words, and throw that into a MultinomialNB classifier. Call fit() and we've got a trained spam filter ready to go! It's just that easy.

In [35]:
vectorizer = CountVectorizer()
counts = vectorizer.fit_transform(data['message'].values)

classifier = MultinomialNB()
targets = data['class'].values
# classifier.fit(counts, targets)

Let's try it out:

In [36]:
examples = ['free car money', "Hi Bob, how about a game of golf tomorrow?"]
example_counts = vectorizer.transform(examples)
predictions = classifier.predict(example_counts)
predictions

NotFittedError: This MultinomialNB instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

## Activity

Our data set is small, so our spam classifier isn't actually very good. Try running some different test emails through it and see if you get the results you expect.

If you really want to challenge yourself, try applying train/test to this spam classifier - see how well it can predict some subset of the ham and spam emails.

In [37]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(counts, targets, test_size=0.2)

In [42]:
X_train.sum()

684740

In [50]:
model = classifier.fit(X_train, y_train)

In [51]:
predictions = classifier.predict(X_test)

In [52]:
model.score(X_test, y_test)

0.9666666666666667